Download monarch-kg sqlite artifact, phenio sqlite artifact and gencc g2d associations

In [1]:
!wget https://data.monarchinitiative.org/monarch-kg/latest/monarch-kg.db.gz
!wget https://data.monarchinitiative.org/monarch-kg/latest/phenio.db.gz
!wget -O gencc.tsv https://search.thegencc.org/download/action/submissions-export-tsv

!gunzip monarch-kg.db.gz
!gunzip phenio.db.gz

--2024-08-13 11:45:53--  https://data.monarchinitiative.org/monarch-kg/latest/monarch-kg.db.gz
Resolving data.monarchinitiative.org (data.monarchinitiative.org)... 35.208.191.193
Connecting to data.monarchinitiative.org (data.monarchinitiative.org)|35.208.191.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3202039360 (3.0G) [application/octet-stream]
Saving to: ‘monarch-kg.db.gz’

monarch-kg.db.gz    100%[===================>]   2.98G  48.5MB/s    in 59s     

2024-08-13 11:46:52 (52.0 MB/s) - ‘monarch-kg.db.gz’ saved [3202039360/3202039360]

--2024-08-13 11:46:52--  https://data.monarchinitiative.org/monarch-kg/latest/phenio.db.gz
Resolving data.monarchinitiative.org (data.monarchinitiative.org)... 35.208.191.193
Connecting to data.monarchinitiative.org (data.monarchinitiative.org)|35.208.191.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2219876009 (2.1G) [application/octet-stream]
Saving to: ‘phenio.db.gz’

phenio.db.gz   

Load gencc associations into monarch-kg.db

In [3]:
!sqlite3 -cmd ".mode tabs" monarch-kg.db ".import gencc.tsv gencc"

Look at the structure of the gencc talbe we just made from the gencc file

In [5]:
!sqlite3 monarch-kg.db ".schema gencc"

CREATE TABLE IF NOT EXISTS "gencc"(
"uuid" TEXT, "gene_curie" TEXT, "gene_symbol" TEXT, "disease_curie" TEXT,
 "disease_title" TEXT, "disease_original_curie" TEXT, "disease_original_title" TEXT, "classification_curie" TEXT,
 "classification_title" TEXT, "moi_curie" TEXT, "moi_title" TEXT, "submitter_curie" TEXT,
 "submitter_title" TEXT, "submitted_as_hgnc_id" TEXT, "submitted_as_hgnc_symbol" TEXT, "submitted_as_disease_id" TEXT,
 "submitted_as_disease_name" TEXT, "submitted_as_moi_id" TEXT, "submitted_as_moi_name" TEXT, "submitted_as_submitter_id" TEXT,
 "submitted_as_submitter_name" TEXT, "submitted_as_classification_id" TEXT, "submitted_as_classification_name" TEXT, "submitted_as_date" TEXT,
 "submitted_as_public_report_url" TEXT, "submitted_as_notes" TEXT, "submitted_as_pmids" TEXT, "submitted_as_assertion_criteria_url" TEXT,
 "submitted_as_submission_id" TEXT, "submitted_run_date" TEXT);


Check out how many GO associations we'll bring in when limiting to just genes in the gencc file (especially to make sure it's not zero!)

In [6]:
!sqlite3 monarch-kg.db "select count(*) from edges where subject in (select distinct gene_curie from gencc) and provided_by = 'go_annotation_edges'"

252959


This loads up both monarch-kg.db and go.db so that associations can be copied over. Here it's taking any association that has a subject in the gencc gene_curie column and a provided_by value that says it came from the go_annotation_edges ingest. (That second constraint isn't the best, but category, predicate & primary_knowledge_source all have a lot of values coming in from this ingest) 

In [8]:
!sqlite3 -cmd "attach 'monarch-kg.db' as monarch" go.db "insert into term_association (id, subject, predicate, object, evidence_type, publication, source) select id, subject, predicate, object, has_evidence as evidence_type, publications as publication, primary_knowledge_source as source from monarch.edges where subject like 'HGNC:%' and object like 'GO:%' and subject in (select distinct gene_curie from monarch.gencc);"


Confirm that go.db now has associations

In [1]:
!sqlite3 -markdown go.db "select predicate, count(*) from term_association group by predicate"

|                     predicate                      | count(*) |
|----------------------------------------------------|----------|
| biolink:active_in                                  | 6227     |
| biolink:actively_involved_in                       | 70456    |
| biolink:acts_upstream_of                           | 282      |
| biolink:acts_upstream_of_negative_effect           | 18       |
| biolink:acts_upstream_of_or_within                 | 1261     |
| biolink:acts_upstream_of_or_within_negative_effect | 7        |
| biolink:acts_upstream_of_or_within_positive_effect | 21       |
| biolink:acts_upstream_of_positive_effect           | 45       |
| biolink:colocalizes_with                           | 474      |
| biolink:contributes_to                             | 591      |
| biolink:enables                                    | 107246   |
| biolink:located_in                                 | 59370    |
| biolink:part_of                                    | 6961     |


From a duckdb instance, export a list of genes and their associated GO terms

In [3]:

!duckdb monarch-kg.duckdb "copy (select subject, string_agg(object,'|') from denormalized_edges where subject_category = 'biolink:Gene' and subject_namespace = 'HGNC' and object_namespace = 'GO' group by all) to 'go_list_per_gene.tsv' (delimiter '\t');"